# Project Background: 
Scenario: A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to. Currently, there is no easy way to query the data to generate the results, since the data reside in a directory of CSV files on user activity on the app.

This ETL flow creates an Apache Cassandra database that we can query to answer questions about song play data. We designed our DB based on questions that the analytics team needs to answer and will be able to test the database by writing queries to answer those questions.

## Notebook Overview:
This script creates an Apache Cassandra database named 'sparkify_db', creates tables that match how we want to model our Sparkify user behavior data, and processes the raw CSV data files to insert the data into our database tables.

This is Project 1B for Udacity Data Engineering Nanodegree program.

# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace/udacity-sparkifydb-apachecassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create and populate Apache Cassandra db

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Create a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_db 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify_db')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra, the database tables must be modeled based on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## For table session_items, the session_id column was used as a partition key because the queries will filter by this column. item_in_session was used as a clustering column to help make up a unique key.
drop_session_items_query = "drop table if exists session_items"
create_session_items_query = """
    create table if not exists session_items (session_id int, 
                                              item_in_session int, 
                                              artist text, 
                                              song text, 
                                              length decimal, 
                                              PRIMARY KEY (session_id, item_in_session))
"""
try:
    session.execute(drop_session_items_query)
    session.execute(create_session_items_query)
except Exception as e:
    print(e)                   

In [9]:
# Parse CSV to insert data into session_items table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into session_items (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
## SELECT statement to verify the data was entered into the table
query = "select artist, song, length from session_items WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2

In [18]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## For table user_sessions, the pair (user_id, session_id) was used as a composite partition key because the queries will filter by these columns. item_in_session was used as a clustering column since we need to sort by that column.
drop_user_sessions_query = "drop table if exists user_sessions"
create_user_sessions_query = """
    create table if not exists user_sessions (user_id int,
                                              session_id int, 
                                              item_in_session int, 
                                              artist text, 
                                              song text, 
                                              user_name text, 
                                              PRIMARY KEY ((user_id, session_id), item_in_session))
"""
try:
    session.execute(drop_user_sessions_query)
    session.execute(create_user_sessions_query)
except Exception as e:
    print(e)        

                    

In [19]:
# Parse CSV to insert data into user_sessions table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into user_sessions (user_id, session_id, item_in_session, artist, song, user_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

In [20]:
## SELECT statement to verify the data was entered into the user_sessions table
query = "select artist, song, user_name from user_sessions WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3

In [21]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## For table song_listeners, the column song was used as a partition key because the queries will filter by this column. user_id was used as a clustering column to make it a unique primary key.
drop_song_listeners_query = "drop table if exists song_listeners"
create_song_listeners_query = """
    create table if not exists song_listeners (song text, 
                                               user_id int,
                                               user_name text, 
                                               PRIMARY KEY (song, user_id))
"""
try:
    session.execute(drop_song_listeners_query)
    session.execute(create_song_listeners_query)
except Exception as e:
    print(e)          

                    

In [22]:
# Parse CSV to insert data into song_listeners table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "insert into song_listeners (song, user_id, user_name)"
        query = query + " VALUES (%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1] + " " + line[4]))

In [23]:
## SELECT statement to verify the data was entered into the song_listeners table
query = "select user_name from song_listeners WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
## Drop the tables before closing out the sessions
try:
    session.execute(drop_session_items_query)
    session.execute(drop_user_sessions_query)
    session.execute(drop_song_listeners_query)
except Exception as e:
    print(e)  

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()